흠.. 일단 내가 아는 선에서 다 해보자.

1. 먼저 데이터 한 번 준비해보자. 희망적인 점은, api 사용해서 json 혹은 txt 형태로 받아올 수 있다는 점이다. json으로 받아오고, dict로 변환한 다음에 필요한 부분만 만들어서.. 일단 대충 이렇게 생각은 든다.

2. 그러면.. 메타데이터 처리는 어떻게 해야할까? 메타데이터 메타데이터 메타데이터.. 메타데이터는 답변의 신뢰도 체감을 높일 수 있는 UX 상의 장치이기도 하다. pdf여러개.. pdf의 페이지도 중요하겠고, 제목도 중요하겠고.. doc loader 및 split 과정에서 이들이 어떻게 처리되는지 확인해 볼 필요가 있다.



API로 정책들을 받아왔을 때, 어떤 것들이 오는지 확인해보았다.
그런 고민이 있었다. 이 답을 그대로 청킹하고 임베딩 하는 순간, 
정책 본문과 일종의 메타데이터로 존재해야 할 제목, 정책 번호 등이 잘리는 것 아닌가?
그렇다면, 이는 별개의 벡터일텐데 의미가 있을까? 다른 어떤 과정이 더 필요하지 않을까?

=> 선별과 조합이 필요하다. 수많은 응답 필드 중 필요한 것들을 뽑아내고, 이를 메타데이터로 삽입하자.
=> metadata가 되어야 할 것들과 content가 되어야할 것들을 구분하자.

이 목록들을 보며 또 다른 고민이 든다.
이 선별을 내가 직접 하는 것이 맞을까?
어느 정보가 어떻게 필요할 줄 알고?
또는 벡터 상에서 어떤 관계를 가질 줄 알고?
머신러닝에서 딥러닝으로 전환되며, feature extracting 없이 말 그대로 거대한 양의 데이터를 떄려넣고 나온 결과물이 지금의 성능이 아닌가?
오히려 이런 방법이 성능을 저하시킬 수 있지 않을까? 하는 고민.

좀 더 생각해보니, 근본적으로 틀렸음을 알 수 있었다.
학습과 활용의 영역. 이는 다르다는 것을.
rag는 모델의 학습 영역이 아니다. 이를 활용하기 위한 방안이고, 따라서 이미 학습을 잘 한 모델에게 가장 정제된 데이터를 넘겨야 좋은 답변을 얻을 수 있다. garbage in garbage out. 이 원칙을 잊고 있었다.


조금만 더 생각해보자.
메타데이터의 역할이 무엇인가?
=> 컨텐츠에 대한 부가적인 정보를 제공함으로써 정확도와 이해도를 높이는 것.

그럼 어떤 것들이 메타데이터로 들어가야할까.. 생각해보자..


일단 모든 출력 결과를 분석해보자
정책 번호 : 사용자가 정책번호로도 검색이 가능하게 됨. 또는 질문 응대 실패 확률이 높을 때? 정책 이름과 번호를 이용한 검증? 같은 것이 가능하지 않을까



In [ ]:
import csv
from langchain_core.documents import Document

def create_documents_from_csv(csv_file_path: str) -> list[Document]:
    """
    CSV 파일에서 정책 정보를 읽어와
    LangChain의 Document 객체 리스트로 변환합니다.

    - 각 row는 하나의 정책 데이터를 나타냅니다.
    - 서술형 정보는 page_content로 조합합니다.
    - 정형 정보는 metadata로 저장합니다.
    """
    
    documents = []
    
    try:
        # UTF-8 인코딩으로 CSV 파일을 엽니다.
        with open(csv_file_path, mode='r', encoding='utf-8') as csvfile:
            # 각 row를 딕셔너리 형태로 읽어옵니다.
            reader = csv.DictReader(csvfile)
            
            for row in reader:
                # 1. page_content 구성: 검색의 대상이 될 자연어 텍스트
                page_content = f"""
정책명: {row.get('plcyNm', '정보 없음')}
정책 설명: {row.get('plcyExplnCn', '정보 없음')}
지원 내용: {row.get('plcySprtCn', '정보 없음')}
신청 자격: {row.get('addAplyQlfcCndCn', '정보 없음')}
소득 조건: {row.get('earnEtcCn', '정보 없음')}
참여 제외 대상: {row.get('ptcpPrpTrgtCn', '정보 없음')}
신청 방법: {row.get('plcyAplyMthdCn', '정보 없음')}
심사 방법: {row.get('srngMthdCn', '정보 없음')}
제출 서류: {row.get('sbmsnDcmntCn', '정보 없음')}
"""

                # 2. metadata 구성: 필터링 및 출처 표시에 사용할 정형 데이터
                metadata = {
                    "policy_id": row.get('plcyNo'),
                    "policy_name": row.get('plcyNm'),
                    "main_department": row.get('sprvsnInstCdNm'),
                    "operating_department": row.get('operInstCdNm'),
                    "business_period": f"{row.get('bizPrdBgngYmd')} ~ {row.get('bizPrdEndYmd')}",
                    "last_updated": row.get('lastMdfcnDt'),
                    "application_url": row.get('aplyUrlAddr'),
                    "reference_url": row.get('refUrlAddr1'),
                    "category_large": row.get('lclsfNm'),
                    "category_medium": row.get('mclsfNm'),
                    "keywords": row.get('plcyKywdNm'),
                    "min_age": row.get('sprtTrgtMinAge'),
                    "max_age": row.get('sprtTrgtMaxAge'),
                    "income_min_amount": row.get('earnMinAmt'),
                    "income_max_amount": row.get('earnMaxAmt'),
                }

                # 신청 기간(aplyYmd)을 시작일과 종료일로 분리하는 로직
                application_period_str = row.get('aplyYmd', '')
                if ' ~ ' in application_period_str:
                    try:
                        start_date, end_date = application_period_str.split(' ~ ')
                        metadata['application_start_date'] = start_date.strip()
                        metadata['application_end_date'] = end_date.strip()
                    except ValueError:
                        metadata['application_period_raw'] = application_period_str
                else:
                    metadata['application_period_raw'] = application_period_str
                
                cleaned_metadata = {k: v for k, v in metadata.items() if v is not None and v != ''}
                
                documents.append(Document(page_content=page_content.strip(), metadata=cleaned_metadata))
                
    except FileNotFoundError:
        print(f"오류: '{csv_file_path}' 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
        
    return documents


In [ ]:
# 1. 함수를 사용하여 Document 리스트 생성
csv_file = "./policy_data.csv"
processed_documents2 = create_documents_from_csv(csv_file)

# 3. 결과 확인
if processed_documents:
    print("--- 첫 번째 Document 객체 ---")
    print("Page Content:")
    print(processed_documents[0].page_content)
    print("\nMetadata:")
    print(processed_documents[0].metadata)

In [ ]:
import json
from langchain_core.documents import Document

def create_documents_from_json(json_file_path: str) -> list[Document]:
    """
    JSON 파일에서 정책 정보를 읽어와
    LangChain의 Document 객체 리스트로 변환합니다.

    - 함수 내부에서 파일을 열고 JSON을 파싱합니다.
    - 서술형 정보는 page_content로 조합합니다.
    - 정형 정보는 metadata로 저장합니다.
    """
    
    documents = []
    
    try:
        # 파일을 열고 JSON 데이터를 파이썬 딕셔너리로 로드
        with open(json_file_path, mode='r', encoding='utf-8') as f:
            data = json.load(f)

        # API 응답 구조에 따라 youthPolicyList에 접근
        policy_list = data.get('result', {}).get('youthPolicyList', [])
        
        if not policy_list:
            return documents

        for policy in policy_list:
            # 1. page_content 구성
            page_content = f"""
정책명: {policy.get('plcyNm', '정보 없음')}
정책 설명: {policy.get('plcyExplnCn', '정보 없음')}
지원 내용: {policy.get('plcySprtCn', '정보 없음')}
신청 자격: {policy.get('addAplyQlfcCndCn', '정보 없음')}
소득 조건: {policy.get('earnEtcCn', '정보 없음')}
참여 제외 대상: {policy.get('ptcpPrpTrgtCn', '정보 없음')}
신청 방법: {policy.get('plcyAplyMthdCn', '정보 없음')}
심사 방법: {policy.get('srngMthdCn', '정보 없음')}
제출 서류: {policy.get('sbmsnDcmntCn', '정보 없음')}
"""

            # 2. metadata 구성
            metadata = {
                "policy_id": policy.get('plcyNo'),
                "policy_name": policy.get('plcyNm'),
                "main_department": policy.get('sprvsnInstCdNm'),
                "operating_department": policy.get('operInstCdNm'),
                "business_period": f"{policy.get('bizPrdBgngYmd')} ~ {policy.get('bizPrdEndYmd')}",
                "last_updated": policy.get('lastMdfcnDt'),
                "application_url": policy.get('aplyUrlAddr'),
                "reference_url": policy.get('refUrlAddr1'),
                "category_large": policy.get('lclsfNm'),
                "category_medium": policy.get('mclsfNm'),
                "keywords": policy.get('plcyKywdNm'),
                "min_age": policy.get('sprtTrgtMinAge'),
                "max_age": policy.get('sprtTrgtMaxAge'),
                "income_min_amount": policy.get('earnMinAmt'),
                "income_max_amount": policy.get('earnMaxAmt'),
            }

            application_period_str = policy.get('aplyYmd', '')
            if ' ~ ' in application_period_str:
                try:
                    start_date, end_date = application_period_str.split(' ~ ')
                    metadata['application_start_date'] = start_date.strip()
                    metadata['application_end_date'] = end_date.strip()
                except ValueError:
                    metadata['application_period_raw'] = application_period_str
            else:
                metadata['application_period_raw'] = application_period_str
            
            cleaned_metadata = {k: v for k, v in metadata.items() if v is not None and v != ''}
            
            documents.append(Document(page_content=page_content.strip(), metadata=cleaned_metadata))

    except FileNotFoundError:
        print(f"오류: '{json_file_path}' 파일을 찾을 수 없습니다.")
    except json.JSONDecodeError:
        print(f"오류: '{json_file_path}' 파일이 올바른 JSON 형식이 아닙니다.")
    except Exception as e:
        print(f"처리 중 오류가 발생했습니다: {e}")
        
    return documents


In [ ]:

# --- 사용 예시 ---


# 2. 함수를 사용하여 Document 리스트 생성
json_file = './policy_data.json'
processed_documents = create_documents_from_json(json_file)

# 3. 결과 확인
if processed_documents:
    print("--- 첫 번째 Document 객체 ---")
    print("Page Content:")
    print(processed_documents[0].page_content)
    print("\nMetadata:")
    print(processed_documents[0].metadata)

In [ ]:
for i in processed_documents:
    print("--- 첫 번째 Document 객체 ---")
    print("Page Content:")
    print(i.page_content)
    print("\nMetadata:")
    print(i.metadata)

In [ ]:
for i in processed_documents2:
    print("--- 첫 번째 Document 객체 ---")
    print("Page Content:")
    print(i.page_content)
    print("\nMetadata:")
    print(i.metadata)

In [ ]:
# 청킹
# 둘 다 해보지 머 .. 원래 AI의 사용은 보통 이런 식이니까
# 1. RecursiveCharacterSplitter
# 2. kss
# 3. 문서 의미 기반 수동 분할 => 우리 데이터의 불규칙한 구조(assorted) 상 불가능.



In [ ]:
# 1. RecursiveCharacterSplitter

from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. RecursiveCharacterTextSplitter 초기화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=100,
    length_function=len, # 글자 수 기준으로 나누겠다.
    is_separator_regex=False,
)

# 2. 준비된 Document 리스트를 분할
split_documents = text_splitter.split_documents(processed_documents2)


# --- 결과 확인 (실제 실행 시 주석 해제) ---
print(f"원본 Document 수: {len(processed_documents2)}")
print(f"분할된 Chunk 수: {len(split_documents)}")

# if split_documents:
#     print("\n--- 첫 번째 분할된 Chunk ---")
#     print("내용:")
#     print(split_documents[0].page_content)
#     print("\n메타데이터:")
#     print(split_documents[0].metadata)
for i in processed_documents2[:10]:
    print("내용:")
    print(i.page_content)
    print("\n메타데이터:")
    print(i.metadata)
    print('--------------------')


In [ ]:
# 2. kss 청킹

import kss


def split_docs_with_kss(documents: list[Document], chunk_size: int = 500) -> list[Document]:
    """
    kss를 사용해 문서를 문장 단위로 분할하고,
    정해진 chunk_size에 맞게 문장들을 다시 그룹화합니다.
    """
    final_chunks = []
    
    for doc in documents:
        # 1. kss로 전체 page_content를 문장 리스트로 분리
        sentences = kss.split_sentences(doc.page_content)
        
        current_chunk_content = ""
        # 2. 문장들을 순회하며 chunk_size에 가깝게 그룹으로 묶기
        for sentence in sentences:
            # 현재 문장을 추가하면 chunk_size를 초과하는지 확인
            if len(current_chunk_content) + len(sentence) > chunk_size and current_chunk_content:
                # 초과한다면, 현재까지의 내용을 하나의 chunk로 만듦
                final_chunks.append(
                    Document(page_content=current_chunk_content.strip(), metadata=doc.metadata)
                )
                # 새로운 chunk 시작
                current_chunk_content = sentence
            else:
                # 초과하지 않으면, 현재 chunk에 문장 추가
                current_chunk_content += " " + sentence

        # 마지막에 남아있는 chunk를 리스트에 추가
        if current_chunk_content:
            final_chunks.append(
                Document(page_content=current_chunk_content.strip(), metadata=doc.metadata)
            )
            
    return final_chunks


In [ ]:

# --- 사용 예시 ---
kss_split_documents = split_docs_with_kss(processed_documents2, chunk_size=500)

# --- 결과 확인 (실제 실행 시 주석 해제) ---
print(f"원본 Document 수: {len(processed_documents)}")
print(f"분할된 Chunk 수 (kss): {len(kss_split_documents)}")

if kss_split_documents:
    print("\n--- 첫 번째 분할된 Chunk (kss) ---")
    print(kss_split_documents[0].page_content)
    print(kss_split_documents[0].metadata)

In [ ]:
# 임베딩 하기
# 프로토 구축이니깐.. 일단 그냥 openai api 사용하자.
# 시간 나면 리더보드 보고 로컬로 해서 결과 비교도 해 보고~



In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS


# 1. OpenAI 임베딩 모델 초기화
# text-embedding-3-small은 비용과 성능 면에서 매우 효율적인 모델입니다.
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")


# 2. 임베딩 및 FAISS 벡터 저장소 생성
# from_documents 함수는 내부적으로 다음 두 가지 작업을 한번에 수행합니다.
#  (1) split_documents의 각 chunk를 embeddings_model을 사용해 벡터로 변환
#  (2) 변환된 벡터와 원본 chunk를 FAISS 데이터베이스에 저장
db = FAISS.from_documents(split_documents, embeddings_model)


# --- 테스트: 유사도 검색 ---
# 생성된 벡터 저장소(db)가 잘 작동하는지 테스트합니다.
query = "미술 분야 청년을 위한 지원 정책 알려줘"
relevant_docs = db.similarity_search(query)


In [ ]:
for i in relevant_docs:
    print(i)
    print('-------------------')

In [ ]:
# page content 동적 생성으로 변경
# 필드가 비었을 경우 그냥 가져가지 않도록.

import csv
from langchain_core.documents import Document

def create_documents_from_csv(csv_file_path: str) -> list[Document]:
    """
    CSV 파일에서 정책 정보를 읽어와
    LangChain의 Document 객체 리스트로 변환합니다.

    - 각 row는 하나의 정책 데이터를 나타냅니다.
    - 서술형 정보는 page_content로 조합합니다.
    - 정형 정보는 metadata로 저장합니다.
    """
    
    documents = []
    
    try:
        with open(csv_file_path, mode='r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            
            for row in reader:
                # --- 이 부분이 수정되었습니다 ---
                # 1. page_content를 동적으로 구성하기 위한 리스트 생성
                content_parts = []
                
                # content에 포함할 필드와 라벨을 정의
                content_fields = [
                    ('plcyNm', '정책명'),
                    ('plcyExplnCn', '정책 설명'),
                    ('plcySprtCn', '지원 내용'),
                    ('addAplyQlfcCndCn', '신청 자격'),
                    ('earnEtcCn', '소득 조건'),
                    ('ptcpPrpTrgtCn', '참여 제외 대상'),
                    ('plcyAplyMthdCn', '신청 방법'),
                    ('srngMthdCn', '심사 방법'),
                    ('sbmsnDcmntCn', '제출 서류')
                ]
                
                # 각 필드에 대해 값이 존재하면 "라벨: 값" 형식으로 리스트에 추가
                for field_key, field_label in content_fields:
                    if row.get(field_key):
                        content_parts.append(f"{field_label}: {row[field_key]}")
                
                # 리스트의 모든 요소를 줄바꿈으로 연결하여 최종 page_content 생성
                page_content = "\n".join(content_parts)
                # ---------------------------------

                # 2. metadata 구성 (이전과 동일)
                metadata = {
                    "policy_id": row.get('plcyNo'),
                    "policy_name": row.get('plcyNm'),
                    "main_department": row.get('sprvsnInstCdNm'),
                    "operating_department": row.get('operInstCdNm'),
                    "business_period": f"{row.get('bizPrdBgngYmd')} ~ {row.get('bizPrdEndYmd')}",
                    "last_updated": row.get('lastMdfcnDt'),
                    "application_url": row.get('aplyUrlAddr'),
                    "reference_url": row.get('refUrlAddr1'),
                    "category_large": row.get('lclsfNm'),
                    "category_medium": row.get('mclsfNm'),
                    "keywords": row.get('plcyKywdNm'),
                    "min_age": row.get('sprtTrgtMinAge'),
                    "max_age": row.get('sprtTrgtMaxAge'),
                    "income_min_amount": row.get('earnMinAmt'),
                    "income_max_amount": row.get('earnMaxAmt'),
                }

                # 신청 기간(aplyYmd) 분리 로직 (이전과 동일)
                application_period_str = row.get('aplyYmd', '')
                if ' ~ ' in application_period_str:
                    try:
                        start_date, end_date = application_period_str.split(' ~ ')
                        metadata['application_start_date'] = start_date.strip()
                        metadata['application_end_date'] = end_date.strip()
                    except ValueError:
                        metadata['application_period_raw'] = application_period_str
                else:
                    metadata['application_period_raw'] = application_period_str
                
                cleaned_metadata = {k: v for k, v in metadata.items() if v is not None and v != ''}
                
                documents.append(Document(page_content=page_content.strip(), metadata=cleaned_metadata))
                
    except FileNotFoundError:
        print(f"오류: '{csv_file_path}' 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
        
    return documents